In [1]:
######################
### INITIALIZATION ###
######################
# mount Google Drive and switch to zatoboj folder in Files view
from google.colab import drive
drive.mount('/content/gdrive')  
from google.colab import files
files.view('/content/gdrive/MyDrive/PUSHMI') 
# automatically mount PyDrive
%cd /content
import json
adc = {                                                                                                                                                                     "client_id": "32555940559.apps.googleusercontent.com", "client_secret": "ZmssLNjJy2998hD4CTg2ejr2", "refresh_token": "1//093X5tMAwUbEGCgYIARAAGAkSNwF-L9IrKTrK23ol4d_X0SIwM2LytDMQfmwR5M9F22I33OXZ_3RthEsqHoXJCSfl5YGSwDKHuSg", "revoke_uri": "https://accounts.google.com/o/oauth2/revoke", "scopes": ["openid","https://www.googleapis.com/auth/userinfo.email","https://www.googleapis.com/auth/cloud-platform","https://www.googleapis.com/auth/appengine.admin","https://www.googleapis.com/auth/compute","https://www.googleapis.com/auth/accounts.reauth","https://www.googleapis.com/auth/drive"],"token_uri": "https://oauth2.googleapis.com/token","type": "authorized_user"}
with open('adc.json', 'w') as f:
    json.dump(adc, f)
from google.colab import auth
auth.authenticate_user()
# if you want to work with git from this notebook
%cd /content/gdrive/My\ Drive/PUSHMI
!mkdir -p ~/.ssh
!cp id_rsa ~/.ssh
!ssh-keyscan -t rsa github.com >> /root/.ssh/known_hosts
!ssh -T git@github.com
!source git_config_startup.sh

Mounted at /content/gdrive


<IPython.core.display.Javascript object>

/content
/content/gdrive/My Drive/PUSHMI
# github.com:22 SSH-2.0-babeld-9d6fcaa2
Hi eduryev! You've successfully authenticated, but GitHub does not provide shell access.


In [17]:
# install module luster
!pip install -q -q -e .
# import module ztpt
import pushmi_pullyu as pushmi
# set up module autoreload
%load_ext autoreload
%autoreload 2
# check your GPU
!nvidia-smi

Mon Jan 11 14:12:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
!git status

fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [4]:
models = luster.train_n_models(num_models=1, batch_size = 300, max_epochs = 50)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type       | Params
------------------------------------------
0 | activation | LeakyReLU  | 0     
1 | layer1     | Sequential | 640   
2 | layer2     | Sequential | 73.9 K
3 | layer3     | Sequential | 295 K 
4 | layer4     | Sequential | 1.2 M 
5 | fc1        | Linear     | 262 K 
6 | fc2        | Linear     | 5.1 K 
------------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/functional_tensor.py:876: UserWarning: Argument fill/fillcolor is not supported for Tensor input. Fill value is zero
  warnings.warn("Argument fill/fillcolor is not supported for Tensor input. Fill value is zero")


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


In [ ]:
model1 = models[0]
d0 = model1.full_eval(mode='train')
predicted_labels = d0['predicted_labels']
true_labels = d0['true_labels']
get_cluster_acc(true_labels, predicted_labels)


0.9623636363636363

In [ ]:
def convert_from_dict(predicted_labels, d):
    for i in d.keys():
        predicted_labels[predicted_labels==i] = 10 + i

    for i in d.keys():
        predicted_labels[predicted_labels==(10 + i)] = d[i]

    return predicted_labels

def convert_predictions(true_labels, predicted_labels):
    counters = []
    convertion_dict = {}
    for i in np.unique(predicted_labels).tolist():
        new_counter = Counter(true_labels[predicted_labels==i])
        counters.append(new_counter)
        right_label = new_counter.most_common(1)[0][0]
        convertion_dict[i] = right_label
    new_predicted_labels = convert_from_dict(predicted_labels, convertion_dict)
    return new_predicted_labels

## TODO: do based on logits, need to permute them according to convert_dict
def combine_predictions(models, mode='val'):
    individual_accs, predictions = [], []
    for model in models:
        d = model.full_eval(mode=mode)
        predicted_labels = d['predicted_labels']
        true_labels = d['true_labels']
        #logits = d['logits']
        individual_accs.append(get_cluster_acc(true_labels, predicted_labels))
        predicted_labels = convert_predictions(true_labels, predicted_labels)
        predictions.append(predicted_labels)
    counters = [Counter([pred[i] for pred in predictions]) for i in range(true_labels.shape[0])]
    final_predictions = np.array([c.most_common(1)[0][0] for c in counters])
    return individual_accs, predictions, true_labels, final_predictions

individual_accs, predictions, true_labels, final_predictions = combine_predictions(models)


In [ ]:
from pprint import pprint
for pred in predictions:
    pprint(pred[final_predictions != true_labels])
pprint(true_labels[final_predictions != true_labels])
print(Counter(true_labels[final_predictions != true_labels]))
print(Counter(true_labels))

array([9, 3, 3, 2, 8, 5, 1, 9, 1, 1, 2, 9, 3, 9, 0, 3, 7, 6, 4, 3, 8, 1,
       3, 3, 9, 2, 7, 3, 2, 2, 1, 8, 2, 9, 9, 3, 9, 2, 3, 9, 2, 2, 6, 0,
       9, 8, 7, 8, 2, 8, 0, 1, 2, 9, 2, 9, 9, 2, 9, 1, 2, 4, 3, 5, 2, 9,
       9, 4, 3, 4, 9, 2, 6, 9, 5, 7, 9, 8, 8, 9, 3, 9, 2, 9, 6, 3, 2, 3])
array([0, 2, 3, 2, 8, 6, 4, 9, 1, 1, 2, 5, 3, 4, 4, 3, 7, 6, 4, 3, 8, 1,
       3, 3, 9, 2, 7, 3, 2, 2, 8, 3, 2, 9, 9, 3, 3, 2, 3, 4, 2, 7, 6, 0,
       7, 8, 7, 6, 2, 6, 0, 2, 2, 3, 5, 3, 7, 2, 7, 1, 2, 4, 3, 3, 2, 4,
       9, 4, 3, 7, 9, 2, 6, 9, 5, 7, 9, 6, 9, 9, 3, 9, 2, 7, 6, 3, 2, 3])
array([0, 2, 9, 2, 8, 6, 4, 9, 1, 1, 2, 5, 5, 4, 6, 9, 7, 6, 4, 9, 1, 1,
       3, 9, 4, 2, 7, 5, 2, 2, 8, 8, 2, 9, 9, 2, 8, 2, 5, 4, 2, 4, 6, 0,
       9, 8, 7, 6, 2, 6, 0, 2, 2, 9, 5, 9, 7, 2, 9, 1, 0, 4, 3, 3, 2, 4,
       9, 4, 5, 4, 8, 2, 6, 9, 5, 7, 9, 6, 1, 1, 3, 1, 2, 7, 6, 3, 2, 3])
array([0, 3, 3, 2, 8, 6, 6, 9, 1, 1, 2, 5, 3, 9, 6, 3, 3, 6, 4, 3, 8, 1,
       3, 3, 9, 2, 7, 3, 2, 2, 8, 3, 2, 9, 9, 3,

In [ ]:
counters = [Counter([pred[i] for pred in predictions]) for i in range(true_labels.shape[0])]

In [ ]:
final_predictions = np.array([c.most_common(1)[0][0] for c in counters])

In [ ]:
final_predictions.shape

(5000,)

In [ ]:
sum(final_predictions[:] != true_labels[:])

233

In [ ]:
4767/5000

0.9534

[]

In [ ]:
i = -1
predicted_labels = val_results[i]['predicted_labels'].cpu().detach().numpy()
true_labels = val_results[i]['true_labels'].cpu().detach().numpy()
acc = val_results[i]['acc']
accs = [x['acc'] for x in val_results]

In [ ]:
accs

[0.1875,
 0.4178,
 0.571,
 0.6298,
 0.6526,
 0.673,
 0.7324,
 0.7834,
 0.8002,
 0.7912,
 0.8142,
 0.8228,
 0.821,
 0.857,
 0.859,
 0.8588,
 0.8764,
 0.8584,
 0.8666,
 0.8658,
 0.8622,
 0.8642,
 0.8652,
 0.864,
 0.8546,
 0.8674,
 0.8714,
 0.8612,
 0.8614,
 0.8576,
 0.8642,
 0.8678,
 0.854,
 0.8776,
 0.8772,
 0.876,
 0.8872,
 0.8852,
 0.898,
 0.9046,
 0.9014,
 0.8956,
 0.8972,
 0.8908,
 0.9006,
 0.9016,
 0.8928,
 0.895,
 0.9024,
 0.8922,
 0.9014,
 0.8998,
 0.895,
 0.8942,
 0.8962,
 0.896,
 0.898,
 0.9002,
 0.9056,
 0.9026,
 0.9108,
 0.9054,
 0.909,
 0.9074,
 0.9016,
 0.902,
 0.902,
 0.9126,
 0.907,
 0.9116,
 0.9114,
 0.9222,
 0.912,
 0.9116,
 0.9098,
 0.9088,
 0.909,
 0.907,
 0.914,
 0.9148,
 0.9144,
 0.9112,
 0.91,
 0.9206,
 0.9168,
 0.9226,
 0.923,
 0.9226,
 0.9228,
 0.932,
 0.9262,
 0.9326,
 0.9312,
 0.938,
 0.9444,
 0.95,
 0.9506,
 0.9552,
 0.9446,
 0.9552,
 0.9568,
 0.9614,
 0.9674,
 0.9682,
 0.97,
 0.9698,
 0.9668,
 0.9696,
 0.9662,
 0.9708,
 0.968,
 0.9702,
 0.9696,
 0.9676,
 0.96

In [ ]:
d1['true_labels'] - d0['true_labels']

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
d1['true_labels'] - val_results[-1]['true_labels'].cpu().detach().numpy()

array([ 0, -2,  4, ...,  4, -4, -2])

In [ ]:
Counter(d0['true_labels']), Counter(val_results[-1]['true_labels'].cpu().detach().numpy())

(Counter({0: 488,
          1: 567,
          2: 506,
          3: 489,
          4: 447,
          5: 465,
          6: 510,
          7: 511,
          8: 513,
          9: 504}),
 Counter({0: 488,
          1: 567,
          2: 506,
          3: 489,
          4: 447,
          5: 465,
          6: 510,
          7: 511,
          8: 513,
          9: 504}))

In [ ]:
Counter(d1['true_labels']), Counter(val_results[-1]['true_labels'].cpu().detach().numpy())

(Counter({0: 466,
          1: 539,
          2: 478,
          3: 505,
          4: 514,
          5: 468,
          6: 499,
          7: 535,
          8: 493,
          9: 503}),
 Counter({0: 488,
          1: 567,
          2: 506,
          3: 489,
          4: 447,
          5: 465,
          6: 510,
          7: 511,
          8: 513,
          9: 504}))

In [ ]:
d1['true_labels'].shape, true_labels.shape

((5000,), (5000,))

In [ ]:
accs

[0.245, 0.6774, 0.335, 0.6458]

In [ ]:
predicted_labels = predicted_labels.cpu().detach().numpy()
true_labels = true_labels.cpu().detach().numpy()

In [ ]:
len(true_labels), len(predicted_labels)

(5000, 5000)

In [ ]:
from collections import Counter
from pprint import pprint
for i in range(10):
    #i = 2
    print(i, Counter(predicted_labels[true_labels==i]), Counter(true_labels[predicted_labels==i]))

0 Counter({6: 465, 7: 1}) Counter({6: 495, 4: 2, 5: 2, 8: 1})
1 Counter({2: 530, 8: 6, 5: 2, 3: 1}) Counter({9: 490, 4: 12, 8: 7, 7: 6, 3: 3, 5: 2, 2: 1})
2 Counter({8: 467, 5: 4, 3: 3, 4: 2, 6: 1, 1: 1}) Counter({1: 530, 4: 3, 8: 1, 9: 1, 6: 1})
3 Counter({3: 498, 1: 3, 5: 2, 7: 1, 4: 1}) Counter({3: 498, 9: 10, 8: 4, 2: 3, 1: 1})
4 Counter({9: 496, 1: 12, 2: 3, 0: 2, 4: 1}) Counter({7: 522, 2: 2, 4: 1, 3: 1})
5 Counter({7: 462, 1: 2, 0: 2, 5: 1, 6: 1}) Counter({8: 475, 2: 4, 9: 2, 1: 2, 3: 2, 6: 2, 5: 1})
6 Counter({0: 495, 5: 2, 9: 1, 2: 1}) Counter({0: 465, 2: 1, 5: 1, 8: 1})
7 Counter({4: 522, 1: 6, 8: 4, 9: 3}) Counter({5: 462, 0: 1, 3: 1})
8 Counter({5: 475, 1: 7, 3: 4, 9: 3, 8: 1, 2: 1, 0: 1, 6: 1}) Counter({2: 467, 1: 6, 7: 4, 8: 1})
9 Counter({1: 490, 3: 10, 5: 2, 2: 1}) Counter({4: 496, 7: 3, 8: 3, 6: 1})


In [ ]:
from collections import Counter
from pprint import pprint
counters = []
for i in range(10):
    #i = 2
    counters.append(Counter(predicted_labels[true_labels==i]))
    print('Breakdown by new classes for true label', i, ': ', Counter(predicted_labels[true_labels==i]))#, Counter(true_labels[predicted_labels==i]))

Breakdown by new classes for true label 0 :  Counter({6: 465, 7: 1})
Breakdown by new classes for true label 1 :  Counter({2: 530, 8: 6, 5: 2, 3: 1})
Breakdown by new classes for true label 2 :  Counter({8: 467, 5: 4, 3: 3, 4: 2, 6: 1, 1: 1})
Breakdown by new classes for true label 3 :  Counter({3: 498, 1: 3, 5: 2, 7: 1, 4: 1})
Breakdown by new classes for true label 4 :  Counter({9: 496, 1: 12, 2: 3, 0: 2, 4: 1})
Breakdown by new classes for true label 5 :  Counter({7: 462, 1: 2, 0: 2, 5: 1, 6: 1})
Breakdown by new classes for true label 6 :  Counter({0: 495, 5: 2, 9: 1, 2: 1})
Breakdown by new classes for true label 7 :  Counter({4: 522, 1: 6, 8: 4, 9: 3})
Breakdown by new classes for true label 8 :  Counter({5: 475, 1: 7, 3: 4, 9: 3, 8: 1, 2: 1, 0: 1, 6: 1})
Breakdown by new classes for true label 9 :  Counter({1: 490, 3: 10, 5: 2, 2: 1})


In [ ]:
sum([max(counter.values()) for counter in counters]) / sum([sum(counter.values()) for counter in counters]), acc

(0.98, 0.98)

In [ ]:
from collections import Counter
from pprint import pprint
counters = []
for i in range(10):
    #i = 2
    counters.append(Counter(predicted_labels[true_labels==i]))
    print('Breakdown by new classes for true label', i, ': ', Counter(predicted_labels[true_labels==i]))#, Counter(true_labels[predicted_labels==i]))

Breakdown by new classes for true label 0 :  Counter({2: 457, 4: 4, 9: 1})
Breakdown by new classes for true label 1 :  Counter({0: 556, 5: 11, 3: 5, 1: 4, 9: 3})
Breakdown by new classes for true label 2 :  Counter({5: 485, 1: 5, 8: 1, 3: 1, 6: 1})
Breakdown by new classes for true label 3 :  Counter({6: 504, 1: 10, 7: 10, 5: 1})
Breakdown by new classes for true label 4 :  Counter({3: 473, 8: 2, 0: 1, 5: 1, 4: 1})
Breakdown by new classes for true label 5 :  Counter({7: 455, 4: 1, 9: 1, 0: 1, 6: 1})
Breakdown by new classes for true label 6 :  Counter({4: 489})
Breakdown by new classes for true label 7 :  Counter({1: 537, 8: 2, 5: 2, 7: 1, 3: 1})
Breakdown by new classes for true label 8 :  Counter({9: 480, 0: 5, 8: 3, 4: 2, 7: 1, 3: 1, 6: 1})
Breakdown by new classes for true label 9 :  Counter({8: 449, 1: 11, 7: 9, 3: 5, 2: 2, 9: 2, 0: 1})


In [ ]:
Counter(predicted_labels), Counter(true_labels)

(Counter({0: 471,
          1: 495,
          2: 488,
          3: 550,
          4: 515,
          5: 513,
          6: 507,
          7: 447,
          8: 446,
          9: 568}),
 Counter({0: 504,
          1: 569,
          2: 480,
          3: 564,
          4: 495,
          5: 451,
          6: 489,
          7: 508,
          8: 477,
          9: 463}))

In [ ]:
logits =  val_results[-1]['logits']

In [ ]:
logits[5:15]

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
        [9.7421e-40, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000

In [ ]:
true_labels[5:15]

array([1, 8, 0, 0, 7, 1, 2, 7, 0, 5])

In [ ]:
model.entropy_loss(torch.mean(logits[:5], axis=0))

tensor(-2.3026, device='cuda:0')